In [20]:
import pandas as pd
from pathlib import Path
import geopandas as gpd
import HydroErr as he

In [4]:
proj_dir = Path("../../..")

In [5]:
test_data = pd.read_csv(proj_dir / "Results/2.WaterTempEst/test_data.csv")

In [26]:
test_data.head()

,Date,Month,DayOfMonth,WaterTemp,LandTemp,NDVI,ClimateClass,Width,ReachID,ReachName,...,dels_scaled,sarea_scaled,rel_dist_scaled,Estimated,variation,type,deviations,width_class,dist_class,deviation
0,2014-07-15,7,15,35.69,41.08,0.37,7,30.0,872,Methow_River_22,...,0.260432,0.034802,0.0,19.119152,1,Linear Regression,-0.250848,Width < 120,>50 km of dam,-0.250848
1,2014-04-01,4,1,10.18,12.33,0.71,26,432.0,992,Pend_Oreille_River_20,...,0.260432,0.034802,0.2,8.989523,1,Linear Regression,2.199523,Width > 120,<50 km of dam,2.199523
2,2013-11-15,11,15,5.66,5.77,0.25,5,190.0,1101,Snake_River_106,...,0.260432,0.034802,0.0,8.982170,1,Linear Regression,-1.737830,Width > 120,>50 km of dam,-1.737830
3,2014-11-01,11,1,10.94,11.95,0.60,9,72.0,596,Willamette_River_2,...,0.260432,0.034802,0.0,13.457792,1,Linear Regression,0.817792,Width < 120,>50 km of dam,0.817792
4,2013-08-01,8,1,26.79,32.26,0.55,9,72.0,596,Willamette_River_2,...,0.260432,0.034802,0.0,17.988932,1,Linear Regression,1.258932,Width < 120,>50 km of dam,1.258932


In [11]:
grand = gpd.read_file(proj_dir / "Data/GIS/shapefiles/CRBDams.shp")

In [25]:
b = test_data.merge(grand[["GRAND_ID", "DEPTH_M"]], left_on="GranD_ID", right_on="GRAND_ID")

In [31]:
b["deviation"] = b["InsituTemp"] - b["Estimated"]
a = b[(b["type"] == "Random Forest Regression") & (b["variation"] == 2)]


In [32]:
shallow = a[a["DEPTH_M"] < 10]
deep = a[a["DEPTH_M"] >= 10]

In [33]:
shallow_rmse = he.rmse(shallow["Estimated"], shallow["InsituTemp"])
deep_rmse = he.rmse(deep["Estimated"], deep["InsituTemp"])

shallow_nse = he.nse(shallow["Estimated"], shallow["InsituTemp"])
deep_nse = he.nse(deep["Estimated"], deep["InsituTemp"])

shallow_r2 = he.r_squared(shallow["Estimated"], shallow["InsituTemp"])
deep_r2 = he.r_squared(deep["Estimated"], deep["InsituTemp"])

In [34]:
print("Shallow RMSE: ", shallow_rmse)
print("Deep RMSE: ", deep_rmse)

print("Shallow NSE: ", shallow_nse)
print("Deep NSE: ", deep_nse)

print("Shallow R2: ", shallow_r2)
print("Deep R2: ", deep_r2)

Shallow RMSE:  4.750314650379208
Deep RMSE:  1.8119924026481462
Shallow NSE:  0.6237091875987684
Deep NSE:  0.8902429793806677
Shallow R2:  0.8397027135325271
Deep R2:  0.8913722923443604


In [38]:
df1 = pd.DataFrame(
    {
        "class": ["Shallow", "Deep"],
        "RMSE": [shallow_rmse, deep_rmse],
        "NSE": [shallow_nse, deep_nse],
        "R2": [shallow_r2, deep_r2],
    },
)

df1.to_csv(proj_dir / "Results/2.WaterTempEst/4.Evaluation/dam_depth_v_performance.csv", index=False)
    